In [ ]:
#this program generates a leaflet map of happiness levels of countries in the world and their population sizes
#Data on 2017 happiness was downloaded from https://www.kaggle.com/unsdsn/world-happiness
#Data on shapefile of countries in the world from http://thematicmapping.org/downloads/world_borders.php
#Data on central coordinates of countries from http://worldmap.harvard.edu/data/geonode:country_centroids_az8
#Data on population of counties from http://worldpopulationreview.com/
#some data processing were done to correct country names like Congo (Kinshasa) to Demo. Republic of Congo and others in excel

In [13]:
#import libs
import pandas as pd
import geopandas as gpd
import pysal as ps
import folium
from fiona.crs import from_epsg
from shapely.geometry import Point
from folium.plugins import MarkerCluster

In [14]:

#define file paths and read into GeoDataFrames Pandas

globcent_fp= r"C:\Users\admin\geopython\L5\EX_Data\country_centroids_az8.csv"
data_fp= r"C:\Users\admin\geopython\L5\EX_Data\2017.csv"
world_fp= r"C:\Users\admin\geopython\L5\EX_Data\world.shp"
pop2018_fp= r"C:\Users\admin\geopython\L5\EX_Data\Glob_pop.csv"


globecent = pd.read_csv(globcent_fp,sep=",",encoding="latin1")

data = pd.read_csv(data_fp,sep=",",encoding="latin1")


pop2018 = pd.read_csv(pop2018_fp,sep=",",encoding="latin1")

world = gpd.read_file(world_fp)






#globecent.columns

#select needed columns of globecent

globecent = globecent[["name_long","name","Latitude","Longitude"]].copy()


#globecent.columns

#data.dtypes

#globecent.dtypes

#world.head()


In [15]:
#create lat and lon fields in data


data = pd.merge(data,globecent,how="inner",left_on="Country",right_on='name_long')

#data.head()

#merge population records with countries shapefile

world= world.merge(pop2018,how="inner",left_on="NAME",right_on='Country')



In [16]:
#create geoid fields and json file

world['geoid'] = world.index.astype(str)
data['geoid'] = data.index.astype(str)



jsontxt_world1 = world.to_json()

In [43]:
#create geometry field in data and populate with points

data['geometry']  = [Point(xy) for xy in zip(data.Longitude , data.Latitude)]

#make geodataframe of data and set crs as same as world

data = gpd.GeoDataFrame(data, crs=world.crs, geometry="geometry")

In [44]:
#create classifier
classifier = ps.Natural_Breaks.make(k=6)

#apply classes to both modes of transportation
world['pop_classes'] = world[["Current Population 2018"]].apply(classifier)



world.head(10)




,FIPS,ISO2,ISO3,UN,NAME,AREA,POP2005,REGION,SUBREGION,LON,...,Laos,Country,Current Population 2018,2030 Population,2050 Population,Growth Rate,Rank,Unnamed: 7,geoid,pop_classes
0,AC,AG,ATG,28,Antigua and Barbuda,44,83039,19,29,-61.783,...,AG,Antigua and Barbuda,103050,114.608,125.223,1.010175,199,1.030500e+08,0,0
1,AG,DZ,DZA,12,Algeria,238174,32854159,2,15,2.632,...,DZ,Algeria,42008054,48821.963,57436.703,1.016698,34,4.200805e+10,1,2
2,AJ,AZ,AZE,31,Azerbaijan,8260,8352021,142,145,47.395,...,AZ,Azerbaijan,9923914,10680.202,11039.198,1.009801,90,9.923914e+09,2,1
3,AL,AL,ALB,8,Albania,2740,3153731,150,39,20.068,...,AL,Albania,2934363,2933.419,2663.591,1.001425,138,2.934363e+09,3,0
4,AM,AM,ARM,51,Armenia,2820,3017661,142,145,44.563,...,AM,Armenia,2934152,2907.463,2700.184,1.001263,139,2.934152e+09,4,0
5,AO,AO,AGO,24,Angola,124670,16095214,2,17,17.544,...,AO,Angola,30774205,44712.282,76046.053,1.033240,46,3.077420e+10,5,2
6,AQ,AS,ASM,16,American Samoa,20,64051,9,61,-170.730,...,AS,American Samoa,55679,57.147,56.759,1.000683,208,5.567900e+07,6,0
7,AR,AR,ARG,32,Argentina,273669,38747148,19,5,-65.167,...,AR,Argentina,44688864,49323.306,55228.902,1.009438,31,4.468886e+10,7,2
8,AS,AU,AUS,36,Australia,768230,20310208,9,53,136.189,...,AU,Australia,24772247,28234.742,33186.818,1.013157,54,2.477225e+10,8,1
9,BA,BH,BHR,48,Bahrain,71,724788,142,145,50.562,...,BH,Bahrain,1566993,2013.135,2327.108,1.049852,152,1.566993e+09,9,0


In [46]:
#drop geom in world and make choropleth maps
c_lat=world['LAT'].mean()
c_lon=world['LON'].mean()

hap_py_map = folium.Map(location=[c_lat, c_lon], tiles='Mapbox Bright', zoom_start=5, control_scale=True, prefer_canvas=True, width=1000, height=1000)


marker_cluster = MarkerCluster().add_to(hap_py_map)

In [47]:
hap_py_map.choropleth(geo_data=jsontxt_world1, data=world, columns=['geoid', 'pop_classes'], key_on="feature.id",
                   fill_color="YlOrRd", fill_opacity=0.9, line_opacity=0.2, line_color='white', line_weight=0,legend_name='Global Population', highlight=False, smooth_factor=1.0)

In [48]:
#create custom icon (happy face smiley)

#mark_fp = "https://photos.app.goo.gl/DK7pvSkw6UCqHg7q7"


#mark=folium.features.CustomIcon(mark_fp, icon_size=5)

#mark
#print (mark)

In [53]:



for idx, row in data.iterrows():
    # Get lat and lon of country centroids 
    lon = row['Longitude']
    lat = row['Latitude']

    # Get happiness rank for popup
    rank = row['Happiness.Rank']
    
    #set the raw score 
    # Get happiness score for marker/icon size
    score = int(row['Happiness.Score'])
    
    # set size with equation size and asign to variable test
    #size_e = (score*2)
    #size = (size_e)
    
    # create tooltip with message Click for rank
    
    tooltip = "Click for Rank"
    
    # Add marker to the map
    
    folium.CircleMarker(location=[lat, lon], radius= 6, popup=str(rank), tooltip=tooltip,color ="#FFFF33",fill_color ="#000080").add_to(hap_py_map)           

    
###folium.Marker(location=[lat, lon], popup=str(rank),icon=(folium.features.CustomIcon(mark_fp, icon_size=test))).add_to(hap_py_map)


# Save the output
outfp = r'C:\Users\admin\geopython\L5\hap.py.map_Isaac.html'
hap_py_map.save(outfp)





In [50]:
world.crs
world.columns
data.columns
data.dtypes

Country                           object
Happiness.Rank                     int64
Happiness.Score                  float64
Whisker.high                     float64
Whisker.low                      float64
Economy..GDP.per.Capita.         float64
Family                           float64
Health..Life.Expectancy.         float64
Freedom                          float64
Generosity                       float64
Trust..Government.Corruption.    float64
Dystopia.Residual                float64
name_long                         object
name                              object
Latitude                         float64
Longitude                        float64
geoid                             object
geometry                          object
dtype: object